<a href="https://githubtocolab.com/giswqs/geemap/blob/master/examples/notebooks/126_selenium_fetch_community_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

## Web retrieval code scripts from google earth engine

- Requires login to google account
- Blocked by default in many current [selenium](https://www.selenium.dev/) drivers
- But is still possible using the [undetected_chromedriver](https://pypi.org/project/undetected-chromedriver/)

Tested on local windows jupyter.

In [1]:
#! pip install geemap
#! pip install selenium
#! pip install undetected_chromedriver

## Open browser

- needs some extras to allow setting download location (ChromeWithPrefs)
- modify prefs download directory as required 

    prefs = {"download.default_directory": r"%\geemap\geemap\data\awesome_gee",}

In [ ]:
prefs = {
        "download.default_directory": r"C:\Users\karel\geemap\geemap\geemap\data\awesome_gee",
        }

In [ ]:
### https://github.com/ultrafunkamsterdam/undetected-chromedriver/issues/524#issuecomment-1058340938

import json
import os
import tempfile
from functools import reduce

import undetected_chromedriver as webdriver


class ChromeWithPrefs(webdriver.Chrome):
    def __init__(self, *args, options=None, **kwargs):
        if options:
            self._handle_prefs(options)

        super().__init__(*args, options=options, **kwargs)

        # remove the user_data_dir when quitting
        self.keep_user_data_dir = False

    @staticmethod
    def _handle_prefs(options):
        if prefs != options.experimental_options.get("prefs"):
            # turn a (dotted key, value) into a proper nested dict
            def undot_key(key, value):
                if "." in key:
                    key, rest = key.split(".", 1)
                    value = undot_key(rest, value)
                return {key: value}

            # undot prefs dict keys
            undot_prefs = reduce(
                lambda d1, d2: {**d1, **d2},  # merge dicts
                (undot_key(key, value) for key, value in prefs.items()),
            )

            # create an user_data_dir and add its path to the options
            user_data_dir = os.path.normpath(tempfile.mkdtemp())
            options.add_argument(f"--user-data-dir={user_data_dir}")

            # create the preferences json file in its default directory
            default_dir = os.path.join(user_data_dir, "Default")
            os.mkdir(default_dir)

            prefs_file = os.path.join(default_dir, "Preferences")
            with open(prefs_file, encoding="latin1", mode="w") as f:
                json.dump(undot_prefs, f)

            # pylint: disable=protected-access
            # remove the experimental_options to avoid an error
            del options._experimental_options["prefs"]    

## Manually log in

In newly opened browser window fill in

- username
- password


Leave the browser open. 

In [ ]:
options = webdriver.ChromeOptions()
options.add_experimental_option("prefs", prefs)

# use the derived Chrome class that handles prefs
driver = ChromeWithPrefs(options=options)
driver.get('https://www.google.com/accounts/Login')

## Retrieve the code examples

In [ ]:
import os
import time

def get_ee_code(uri='https://code.earthengine.google.com/scripts/public/load?id=7592ab36de453d718aab7291692abb88', label='test_example'):
    
    code = uri.replace('https://code.earthengine.google.com/scripts/public/load?id=','')
    old_name = "f.json"
    new_name = f"{code}.json"
    old = os.path.join(prefs["download.default_directory"], old_name)
    new = os.path.join(prefs["download.default_directory"], new_name)
    
    if os.path.isfile(old):
        os.remove(old)
    
    driver.get(uri)
    time.sleep(1)  
    
    counter = 0
    while not os.path.isfile(old) and counter < 12:
        time.sleep(1)
        counter += 1
        
    if os.path.isfile(new):
        # print(f"The file already exists: {code[:4]}, {label}")
        os.remove(new)       
    os.rename(old, new)


In [ ]:
get_ee_code()

## Go get a cup of coffee / tea

- attempts to fetch each code example from community datasets
- plus 900 datasources (with duplicates, 140 unique ones)
- takes about 1-3 seconds per page. 
- not all can be loaded (f.e. those using https://earthenv-dot-map-of-life.appspot.com/)

In [ ]:
import geemap

def get_community_examples():
    assets = geemap.search_ee_data('', source='community')
    for asset in assets:
        asset['uri'] = asset['sample_code'].replace('https://code.earthengine.google.com/',
                                  'https://code.earthengine.google.com/scripts/public/load?id=')
    return assets

seen = set()

for ix, (x,y) in enumerate([(asset['uri'], asset['title']) for asset in get_community_examples()]):
    if not ix%20:
        print(ix)
    try:
        if x not in seen:
            get_ee_code(x,y)
            seen.add(x)
    except:
        print(f"Failed loading code for {x}, {x}")

## Update to repository

https://github.com/giswqs/geemap